# Mixed-integer programming: Tools and tricks

Operations research often involves models where you need discrete decisions. 

Oftentimes, this will lead to problems that are $NP$-hard. What do you do?

1. Give up
2. Settle for approximate solutions
2. Try the simplest thing imaginable: **enumeration**

Consider the simple knapsack. Iain wants to carry items to the pawn shop to get some extra cash. He has $N$ items, each with a weight $w_i$ and a price $p_i$. Iain hasn't been to the gym lately, so he can only carry $C$ kilos. How does he choose what to bring with him?

We can model this as an integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N p_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \{0,1\} \quad \forall i = 1,\ldots,N
\end{align*}

How would you solve this? The simple way is just to consider each possible value for $x$ and compare the cost. After Iain has weighed all $2^N$ possible collections of items (and verified that he can lift them at once), he just chooses the best set.

We can visualize this approach to enumeration like this:
![Enumerate](http://www.mit.edu/~huchette/.tmp/enumerate.svg)

But again, we don't want to _actually_ enumerate every solution. Bear with me while I construct a binary tree like so:
![Full Tree](http://www.mit.edu/~huchette/.tmp/fulltree.svg)
It's rooted at what we call the _relaxation_: none of variables have integrality enforced. As we go down leaves of the tree, we add pick a variable to _branch_ on, and create two descended nodes that fix that variable to one of its possible values. If we follow the tree all the way to the bottom, we reach our enumeration from before.

Again, this is even more complexity, and what does it buy us? Well, with the tree structure in place, we can make a simple observation about bounding. Since as we go down the arcs of the tree we restrict our problem more and more, we must have that:

>If node ``q`` is descended from node ``p``, we must have that the optimal cost of subproblem ``q`` is no more than that for node ``p``

This leads us to a powerful tool in solving these enumeration problems: 

>If I can show you that the optimal cost for subproblem ``q`` is _less_ than the optimal cost for the original problem, the same is true for any descendent of ``q``. 

That is, we can _prune_ the tree and safely discard some nodes, kind of like this:
![Pruning](http://www.mit.edu/~huchette/.tmp/fathom.svg)

## Example

Consider the following (small) knapsack problem:

\begin{align*}
    \max\quad& x + y \\
    \text{s.t.}\quad& x + 2y \leq 1.5 \\
    & 0 \leq x, y \leq 1 \\
    & x, y \in \{0,1\}
\end{align*}

What does the relaxation (no integrality restrictions) of this problem look like?

![fig1](http://www.mit.edu/~huchette/.tmp/fig1.svg)

First, we solve the LP relaxation and get $(x^*,y^*) = (1,0.25)$. 

![fig1](http://www.mit.edu/~huchette/.tmp/fig2.svg)

This isn't integer feasible, so we branch on $y$. The subproblem with $y = 1$ is infeasible, and the subproblem with $y = 0$ is feasible with solution $(x^*,y^*) = (1,0)$. This is integer feasible, so we update our lower bound. We've also exhausted the tree, so we have our optimal solution!

![fig1](http://www.mit.edu/~huchette/.tmp/fig3.svg)

In the worst case scenario, the branch-and-bound scheme can end up solving even _more_ subproblems than if we just enumerated everything to begin with. For the scheme to work well, we need to prune large portions of the tree. 

## Branch-and-bound
We'll keep track of a global _lower bound_ $LB$ for our problem. Each node ``q`` will have an upper bound $UB_q$ that it inherents from its parent. If we get to the point where we have solved all subproblems (or, ideally, pruned off a great deal of them), we know that we're optimal. To do this we'll also keep track of a list $L$ of subproblems left to solve; initially, it's just the relaxation. The procedure is:

While $L$ is not empty, pick a subproblem ``q`` out of our list $L$ and solve it. 
1. ``if`` ``q`` is infeasible, ``continue``
2. ``if`` the solution is integer feasible, update the lower bound $LB$ if the cost is higher than what we had before
3. ``if``  the relaxation value is less than our global $LB$ ``continue``
4. ``else`` pick a non-integer variable $i$ and _branch_ by adding two subproblems to $L$: 
    * One with $x_i = 0$
    * Another with $x_i = 1$

Branch-and-bound is sometimes called an _implicit enumeration_ scheme because of step 3: we avoid solving any subproblems that we can prove won't produce the optimal solution.

** The "magic" of modern MIP solvers largely comes down to pruning massive portions of the tree **

## Things you maybe don't need to know about (but are cool anyway)

1. Solvers _warm start_ using dual simplex: old optimal solutions are used as starting points at nodes, reducing number of simplex pivots required.
2. Solvers usually will refine the root node for a while, adding cuts and such repeatedly. We'll see this below in solver output.

## Stuff you should care about

1. Cuts (improve the _upper bounds_)
2. Heuristics (improve the _lower bounds_)
3. Presolve (_simplify_ everything before we build the tree)
4. Branching strategies (construct the tree in a smart way)

## 1. Cuts

Cuts are inequalities that are _valid_ (don't cut off any feasible integer points) and _strengthen_ the formulation (chop off some of the region feasible for the relaxation)

Three main types of cuts:

1. General purpose (e.g. CG, split, MIR)
2. Structure-specific (e.g. knapsack cover, clique)
3. Problem-specific (whatever _you_ cook up)

### Example 

Let's go back to our knapsack example and imagine that had been a bit smarter and realized that $x + \frac{4}{3}y \leq 1$ is feasible for all integer feasible points. If we add this to our formulation, we have the optimal solution at the root node!

![fig1](http://www.mit.edu/~huchette/.tmp/fig4.svg)

## 2. Heuristics

You can add integer feasible solutions into the branch-and-bound scheme to improve the lower bound. These can come from

1. Problem-specific heuristics
2. Neighborhood search
3. Rounding or "polishing"

### Example

After we've already solved it twice, it's easy to see that the optimal solution for our knapsack is $(1,0)$. So, we can just start out the branch-and-bound procedure with $LB = 1$.

More generally, if we're solving a knapsack problem, one simple heuristic is to add a _greedy solution_ where you iteratively add the best available item to the sack until you run out of room. This will often be a very good solution, and is a simple example of a problem-specific heuristic scheme.

![fig5](http://www.mit.edu/~huchette/.tmp/fig5.svg)

## 3. Presolve

Modern MIP solvers have a litany of techniques to simplify problems _before_ constructing the tree. 

* Variable/constraint bounds tightening
* Logical inferences
* General cleanup (remove redundant variables, constraints)

By shrinking problems, presolve potentially shrinks

* the B&B tree (fewer binary variables)
* The node solve time (smaller LP relaxations)

### Example

For our problem
\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1.5 \\
    & x, y \in \{0,1\}
\end{align*}

we can perform bounds tightening to get

\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1 \\
    & x, y \in \{0,1\}
\end{align*}

![fig1](http://www.mit.edu/~huchette/.tmp/fig6.svg)

Then, we can reason that, since $x,y \in \{0,1\}$, we cannot have $y=1$, and so we can reduce our problem to 

\begin{align*}
    \max\:& x \\
    & x \in \{0,1\}
\end{align*}

Which we can then solve in closed form.

## 4. Branching strategies

Different paths through the B&B tree can see drastically different performance:

* Missing a node that produces a good feasible solution will mean you add nodes to your list you otherwise would have pruned
* Choosing nodes that would otherwise be pruned just slows you down

How do we know _a priori_ what a good branching strategy is, though? 

We don't really, but you can make informed guesses. 

Consider another small problem:
\begin{align*}
    \max\:& y \\
    \text{s.t.}\:& -x + y \leq \frac{1}{3} \\
    & x + y \leq \frac{4}{3} \\
    & 0 \leq x,y \leq 1 \\
    & x, y \in \{0,1\}
\end{align*}
If we solve the relaxation, we get an optimal solution of $(x^*,y^*) = (\frac{1}{2},\frac{5}{6})$.

![fig1](http://www.mit.edu/~huchette/.tmp/fig7.svg)

Now we have a choice: both coordinates are fractional, so we can branch on either. If we branch on $y$, we get

![fig1](http://www.mit.edu/~huchette/.tmp/fig8.svg)

The branch with $y=0$ gives us an optimal solution in $(0,1)$, and the $y=1$ branch is infeasible, so we have no more branches to take and we are done. But if we had branched on $x$ instead,

![fig1](http://www.mit.edu/~huchette/.tmp/fig9.svg)

The optimal solution for both branches is still fractional, so we have to branch again. If we have, say, 1000 binary variables instead of just two, we see how branching strategies can have an enormous effect on performance.

## Conclusion
MIP solvers are magical in two ways:

1. They can be incredibly effective
2. They can be incredibly opaque

# Understanding solver output

Let's solve our simple knapsack problem for Iain and see what the solver spits out.

In [ ]:
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

@defVar(m, x, Bin)
@defVar(m, y, Bin)

@addConstraint(m, x + 2y ≤ 1.5)
@setObjective(m, Max, x + y)

solve(m)

What's going on here? First, Gurobi is telling us some summary statistics about our problem:
```
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+00]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e+00, 2e+00]
```
Next, we see that Gurobi ran some heuristic procedure _before_ constructing the tree, and produced a feasible solution with value 1:
```
Found heuristic solution: objective 1
```
Next, Gurobi runs presolve and removes 1 row and 2 columns: in other words, it removes everything! (It also does this really quickly).
```
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
```
Since there isn't anything left to the problem, it doesn't have to look at any nodes, and it does this about as fast as you would expect:
```
Explored 0 nodes (0 simplex iterations) in 0.00 seconds
```
Gurobi is done solving the problem, so now it tells us some summary statistics: the objective value of the best feasible solution, the best upper bound, and the percent gap between the two:
```
Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0%
```

This is kind of dull since Gurobi solves this before we ever get to the branch-and-bound tree! Let's cook up a problem that's a little more interesting. What about more items, and more knapsacks! If $N=350$, naïve enumeration would create $2^{350}$ nodes, which would take quite some time. How does the solver actually tackle it?

In [ ]:
srand(100)
N = 350

m = Model(solver=GurobiSolver())
@defVar(m, x[1:N], Bin)
for _ in 1:10
    @addConstraint(m, dot(rand(N), x) ≤ N / 50)
end

@setObjective(m, Max, dot(rand(N), x))

solve(m)

The junk at the top is mostly the same, but now we force the solver to actually do some work. First, it finds an alright heuristic solution:
```
Found heuristic solution: objective 6.86518
```
Presolve isn't able to do much of anything (probably because the problem is dense):
```
Presolve time: 0.00s
Presolved: 10 rows, 350 columns, 3500 nonzeros
Variable types: 0 continuous, 350 integer (350 binary)
```
Then it solves the relaxation and reports back:
```
Root relaxation: objective 1.599653e+01, 33 iterations, 0.00 seconds
```
Now it explores the branch-and-bound tree, and updates us as it goes along:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.99653    0   10    6.86518   15.99653   133%     -    0s
H    0     0                      15.0893488   15.99653  6.01%     -    0s
H    0     0                      15.4881914   15.99653  3.28%     -    0s
     0     0   15.99099    0   13   15.48819   15.99099  3.25%     -    0s
     0     0   15.99099    0   10   15.48819   15.99099  3.25%     -    0s
     0     0   15.99099    0   13   15.48819   15.99099  3.25%     -    0s
     0     0   15.99084    0   13   15.48819   15.99084  3.25%     -    0s
     0     2   15.99084    0   13   15.48819   15.99084  3.25%     -    0s
H52632 17202                      15.4995833   15.75331  1.64%   3.9    3s
 72254 22464   15.71500   38    9   15.49958   15.72628  1.46%   3.9    5s
H101962 27634                      15.5156084   15.69895  1.18%   3.9    6s
 172259 28853   15.62486   37   10   15.51561   15.64580  0.84%   3.8   10s
```
What does this all mean? Let's look at just the first line:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.99653    0   10    6.86518   15.99653   133%     -    0s
```
We see that the information is broken down into four main columns:

1. ``Nodes``: Global node information
    * how many nodes have we looked at
    * how many do we have in our queue
2. ``Current Node``
    * objective
    * depth in the tree
    * number of noninteger variables in the solution
3. ``Objective Bounds``
    * Best incumbent (lower bound)
    * node upper bound
    * the gap between the two
4. ``Work``
    * average simplex iterations per node
    * total elapsed time

Finally, we get a neat summary of the cutting planes Gurobi found useful:
```
Cutting planes:
  Gomory: 10
  Cover: 5
```
All told, we explored 257,956  nodes, much less than the $2^{100}$ we were worried about. All this only took 951,664 simplex iterations and 13.70 seconds, which shows the power of _warm starting_ in the branch-and-bound scheme.

Now what about those ``H``s that appear? That tells us that Gurobi ran a heuristic and found a new best solution. You can see for yourself, as the incumbent value increases while the bound remains the same; we also don't get any ``Current Node`` information:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.99653    0   10    6.86518   15.99653   133%     -    0s
H    0     0                      15.0893488   15.99653  6.01%     -    0s
```
You'll also sometimes see a ``*`` instead of the ``H``, which says that the feasible solution came from branching instead of heuristics.

Gurobi likes to spare your screen, so it doesn't dump information about every node, but will update you periodically as it works through the tree.

# Solver parameters: Should you bother?

Gurobi (and other high-quality solvers) allow you to tweak a wide range of different parameters; _sometimes_ tuning these can drastically improve performance. It can be kind of intimidating, though: Gurobi has over 100 parameters, so which are the important ones?

Some useful ones:

* ``TimeLimit``: how long the solver will run before giving up
* ``NodeLimit``: how many nodes to explore before giving up
* ``MIPGap``: termination criterion for relative gap $\frac{UB-LB}{LB}$
* ``MIPFocus``: High-level controls on solver priority (proving optimality or increasing bound or finding optimal solution)

Is that it? Well, no, but you probably need domain knowledge about your problem to go much further. There's an alternative: Gurobi has a parameter tuning feature you can try to "learn" good parameter settings for a particular model. Try it out if you aren't quite happy with your performance.